# Studi Kasus Bukit Darmo Property

# PROSES PERAMALAN MODEL SAMPAI TAHUN 2025 JUNI

## PROFILE KELOMPOK


NPM KETUA : 200411100153

NAMA KETUA : FIKRI AINUN NAJIB

NPM ANGGOTA : 200411100166

NAMA ANGGOTA : GHANIY ALBAR RASYIDI KUSUMA

GITHUB : https://github.com/LALA09-erha/Streamlit-bukitdarmoproperty-UAS

DATASET : https://finance.yahoo.com/quote/BKDP.JK/history?p=BKDP.JK


KELAS PSD C

Inisialisasi Libary dan beberapa Model

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from numpy import array
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
import datetime
from dateutil.relativedelta import relativedelta
from sklearn.tree import DecisionTreeRegressor

from sklearn.decomposition import PCA



Pengambilan data

In [14]:
data = pd.read_csv("../Data/BKDP.JK.csv")

# drop Date column from data
data = data['Volume']



Split Data

In [15]:
data_training,data_test = train_test_split(data, train_size=0.8, test_size=0.2, shuffle=False, random_state=42)

In [16]:
data_training_new = pd.DataFrame(data_training)
data_test_new = pd.DataFrame(data_test)

fungsi Cross Validation

In [17]:
# Cross Validation
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  
  return array(X), array(y)
# define univariate time series

Eksekusi Export Data Ke CSV

In [18]:
# PROSES MEMASUKKAN DATA PREDIKSI KE CSV
# membuat dataframe kosong dengan 3 kolom
data_final = pd.DataFrame(columns=['Date','Volume'])
# berawal pada 1995-september
start_date = datetime.date(2023, 6, 16)
 
# berakhir pada 2023-januari 
end_date = datetime.date(2026, 6, 16)

# loop setiap bulan
delta = relativedelta(days=1)
while start_date <= end_date:
  n = 4
  tanggal = start_date.strftime("%Y-%m-%d")
  # Preprosssing data
  sc = MinMaxScaler()
  train_scaled = sc.fit_transform(data_training_new)
  test_scaled = sc.transform(data_test_new)

  # Proses Pemanggilan Cross Validation 
  X_train, y_train = split_sequence(train_scaled, n) 
  X_test, y_test = split_sequence(test_scaled, n) 
  # membuat kolom otomatis
  colom = []
  for c in range(n):
      if(c==0):
          colom.insert(0,"Xt")
      else:
          colom.insert(0,f"Xt-{c}")



  # # Masukkan ke dataframe hasil dari split  squens
  # UNTUK MEMISAHKAN X TRAIN, YTRAIN dan TEST
  newX_train = pd.DataFrame(np.reshape(X_train, (len(X_train) , n)),columns=colom)
  newY_train = pd.DataFrame(y_train, columns=['Output'])
  new_data_train = pd.concat([newX_train,newY_train],axis=1)
  newX_test = pd.DataFrame(np.reshape(X_test, (len(X_test) , n)),columns=colom)
  newY_test = pd.DataFrame(y_test, columns=['Output'])
  new_data_test = pd.concat([newX_test,newY_test],axis=1)

  # EKSEKUSI KE model


  #  Predict dengan KNN
  knn = KNeighborsRegressor(n_neighbors=27)
  knn.fit(newX_train, newY_train)
  y_pred_knn = knn.predict(newX_test)
  y_pred_knn_shape = y_pred_knn.reshape(-1,1)
  y_pred_knn_inverse = sc.inverse_transform(y_pred_knn_shape)

  # mean dari 3 model dimasukkan ke dataframe data_test_new 
  data_final = pd.concat([data_final,pd.DataFrame({'Date':tanggal,'Volume':[int(np.round(y_pred_knn_inverse[-1][0]))]})],axis=0)
  data_test_new = pd.concat([data_test_new,pd.DataFrame({'Volume':np.round(np.mean([(y_pred_knn_inverse[-1])]))},index=  [0] )],axis=0)
  start_date += delta

# save data to csv
data_final.to_csv('../Data/prediksijanuari2023.csv', index=False)
data_test_new.to_csv('../Data/zatatestjanuari2023.csv', index=False)

